# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,susuman,62.7833,148.1667,-19.81,97,100,0.20,RU,1729541744
1,1,badger,64.8000,-147.5333,2.14,91,100,1.54,US,1729541745
2,2,grytviken,-54.2811,-36.5092,2.15,96,100,2.43,GS,1729541746
3,3,namibe,-15.1961,12.1522,19.91,79,5,1.82,AO,1729541747
4,4,saint-yrieix-la-perche,45.5160,1.2057,15.03,93,76,1.51,FR,1729541748


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.9,
    alpha = 0.7,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 24) & (city_data_df["Max Temp"] > 18) & (city_data_df['Cloudiness'] == 0), :]

# Drop any rows with null values
ideal_city_data_df.dropna(how="any")

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
113,113,tura,25.5198,90.2201,20.91,89,0,2.17,IN,1729541876
144,144,bethanie,-26.4833,17.1500,18.48,20,0,4.31,NaN,1729541918
149,149,pisco,-13.7000,-76.2167,22.03,64,0,7.20,PE,1729541923
169,169,nantucket,41.2835,-70.0995,20.06,60,0,6.69,US,1729541920
206,206,carnarvon,-24.8667,113.6333,18.19,69,0,11.72,AU,1729541990
211,211,el granada,37.5027,-122.4694,21.91,67,0,5.66,US,1729541996
228,228,pimentel,-6.8367,-79.9342,22.16,64,0,9.26,PE,1729542018
232,232,outjo,-20.1167,16.1500,20.46,14,0,4.22,NaN,1729542023
261,261,giridih,24.1833,86.3000,22.88,69,0,1.75,IN,1729542057
262,262,lompoc,34.6391,-120.4579,22.85,43,0,7.72,US,1729542058


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
113,tura,IN,25.5198,90.2201,89,
144,bethanie,NaN,-26.4833,17.1500,20,
149,pisco,PE,-13.7000,-76.2167,64,
169,nantucket,US,41.2835,-70.0995,60,
206,carnarvon,AU,-24.8667,113.6333,69,
211,el granada,US,37.5027,-122.4694,67,
228,pimentel,PE,-6.8367,-79.9342,64,
232,outjo,NaN,-20.1167,16.1500,14,
261,giridih,IN,24.1833,86.3000,69,
262,lompoc,US,34.6391,-120.4579,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "filter":"",
    "bias":"",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: Hotel Polo Orchid
bethanie - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
nantucket - nearest hotel: Faraway Nantucket
carnarvon - nearest hotel: No hotel found
el granada - nearest hotel: Beach House
pimentel - nearest hotel: Hospedaje Calle del Sol
outjo - nearest hotel: Ombinda Country Lodge
giridih - nearest hotel: No hotel found
lompoc - nearest hotel: Red Roof Inn Lompoc
pacific grove - nearest hotel: Pacific Grove Inn
fredericton - nearest hotel: Days Inn
khuzdar - nearest hotel: Al-Mustafa hotel
valparaiso - nearest hotel: Ibis Hotel
alexandria - nearest hotel: Smoha Zahran Haus (7th floor)
dauphin - nearest hotel: Best Western Plus Dauphin
monterey - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
eastham - nearest hotel: Four Points by Sheraton
daultala - nearest hotel: No hotel found
sainte-marie - nearest hotel: Hôtel Select Océan Indien
al jawf - nearest hotel: No hotel found
molina - 

,City,Country,Lat,Lng,Humidity,Hotel Name
113,tura,IN,25.5198,90.2201,89,Hotel Polo Orchid
144,bethanie,NaN,-26.4833,17.1500,20,No hotel found
149,pisco,PE,-13.7000,-76.2167,64,La Portada
169,nantucket,US,41.2835,-70.0995,60,Faraway Nantucket
206,carnarvon,AU,-24.8667,113.6333,69,No hotel found
211,el granada,US,37.5027,-122.4694,67,Beach House
228,pimentel,PE,-6.8367,-79.9342,64,Hospedaje Calle del Sol
232,outjo,NaN,-20.1167,16.1500,14,Ombinda Country Lodge
261,giridih,IN,24.1833,86.3000,69,No hotel found
262,lompoc,US,34.6391,-120.4579,43,Red Roof Inn Lompoc


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.9,
    alpha = 0.7,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)